# ABS Monthly Consumer Price Index Indicator 6484

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt

# local imports
from abs_data_capture import (
    get_fs_constants,
    get_ABS_meta_and_data,
    get_plot_constants,
    find_id,
    get_meta_constants,
)
from plotting import (
    finalise_plot,
    set_chart_dir,
    clear_chart_dir,
    plot_growth,
    line_plot,
    COLOR_BLUE,
    COLOR_AMBER,
)

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

## Get data from ABS

In [2]:
catalogue_id = "6484"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)
abs_dict = get_ABS_meta_and_data(catalogue_id, verbose=False)
if abs_dict is None:
    sys.exit(-1)
meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Cache looks stale: Removing old cache version
We need to download this file from the ABS ...
Saving ABS download to cache.
Extracting DataFrames from the zip-file ...


## Plot

### Set-up 

In [3]:
plt.style.use("fivethirtyeight")
set_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

### Functions

In [4]:
def annualise_percentages(series: pd.Series, periods: int = 12) -> pd.Series:
    """Annualise a growth rate for a period."""

    s = series / 100.0
    annual = (((1 + s) ** periods) - 1) * 100
    return annual

### Plot growth - combined line/bar charts

In [5]:
series_type = "Seasonally Adjusted"
change = "Percentage Change"
annual = "Corresponding Month of Previous Year"
monthly = "from Previous Period"

for series_name in ["All groups CPI", "Trimmed Mean"]:
    if not meta[did_col].str.contains(series_name).any():
        continue

    data = []
    for period in annual, monthly:
        selector = {
            series_type: type_col,
            period: did_col,
            series_name: did_col,
            change: did_col,
        }
        series_id, units = find_id(meta, selector, verbose=False)
        table = meta.loc[meta["Series ID"] == series_id, "Table"].values[0]
        data.append(abs_dict[table][series_id])

    ax = plot_growth(*data)
    if ax:
        ax.axhspan(2, 3, color="#dddddd", zorder=-1, label="Annual inflation target")
        ax.legend(loc="best", fontsize="small")
        finalise_plot(
            ax,
            title=f"Monthly CPI Indicator - Growth - {series_name}",
            ylabel="Per cent",
            rfooter=f"{source}",
            lfooter=f"Australia: {series_type.capitalize()} series. ",
            # show=True,
        )

### Selected sub items

In [6]:
selectors = {
    "Food and non-alcoholic beverages": {
        "Food and non-alcoholic beverages": did_col,
        "Original": type_col,
        annual: did_col,
    },
    "Automotive fuel": {
        "Automotive fuel": did_col,
        "Original": type_col,
        annual: did_col,
    },
}

data = pd.DataFrame()
for name, selector in selectors.items():
    id, units = find_id(meta, selector, verbose=False)
    table = meta.loc[meta["Series ID"] == id, "Table"].values[0]
    data[name] = abs_dict[table][id]

title = "Monthly CPI Indicator - Annual Growth - Food & Fuel"
tag = "1"
series_type = "Original series"
line_plot(
    data,
    title=title,
    ylabel="Per cent",
    color=[COLOR_BLUE, COLOR_AMBER],
    width=1.5,
    tags=tag,
    legend={"loc": "best", "fontsize": "x-small"},
    rfooter=f"{source}",
    lfooter=f"Australia: {series_type}",
    # show=True,
)

In [7]:
selectors = {
    "Rents": {
        "Rents": did_col,
        "Original": type_col,
        annual: did_col,
    },
    "New dwelling purchase by owner-occupiers": {
        "New dwelling purchase by owner-occupiers": did_col,
        "Original": type_col,
        annual: did_col,
    },
}

data = pd.DataFrame()
for name, selector in selectors.items():
    id, units = find_id(meta, selector, verbose=False)
    table = meta.loc[meta["Series ID"] == id, "Table"].values[0]
    data[name] = abs_dict[table][id]

title = "Monthly CPI Indicator - Annual Growth - Residential"
tag = "2"
series_type = "Original series"
line_plot(
    data,
    title=title,
    ylabel="Per cent",
    color=[COLOR_BLUE, COLOR_AMBER],
    width=1.5,
    tags=tag,
    legend={"loc": "best", "fontsize": "x-small"},
    rfooter=f"{source}",
    lfooter=f"Australia: {series_type}",
    # show=True,
)

## Finished

In [8]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Fri Mar 10 2023 20:49:15

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.11.0

sys       : 3.11.0 | packaged by conda-forge | (main, Jan 15 2023, 05:44:48) [Clang 14.0.6 ]
pandas    : 1.5.3
matplotlib: 3.7.1

Watermark: 2.3.1



In [9]:
print("Done")

Done
